USing VGG16 pretrained model

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.applications.vgg16 import VGG16 as PretrainedModel, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys, os, random

In [2]:
!ls

drive  sample_data  test  train  validation


In [3]:
train_data_path = "train"

In [4]:
labels = os.listdir(train_data_path)
labels

['c4', 'c9', 'c6', 'c1', 'c2', 'c8', 'c3', 'c5', 'c7', 'c0']

In [5]:
img_size = [224, 224]

In [6]:
image_files = glob(train_data_path + '/*/*.jpg')

In [7]:
folders = glob(train_data_path + '/*')
folders

['train/c4',
 'train/c9',
 'train/c6',
 'train/c1',
 'train/c2',
 'train/c8',
 'train/c3',
 'train/c5',
 'train/c7',
 'train/c0']

In [8]:
ptm = PretrainedModel(
            input_shape = img_size + [3],
            weights = 'imagenet',
            include_top = False)

In [9]:
ptm.trainable = False

In [10]:
K = len(folders)
x = Flatten()(ptm.output)

#using softmax because we are not having binary classification
x = Dense(K, activation = 'softmax')(x)

In [11]:
model = Model(inputs = ptm.input, outputs = x)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
#instance of image data generator
gen = ImageDataGenerator(
      validation_split = 0.2,
      preprocessing_function = preprocess_input)

In [14]:
batch_size = 32

#create generators
train_generator = gen.flow_from_directory(
train_data_path,
shuffle = False,
target_size = img_size,
batch_size = batch_size,
subset = 'training'
)

valid_generator = gen.flow_from_directory(
train_data_path,
subset = 'validation',
target_size = img_size,
batch_size = batch_size,
)

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [15]:
#compiling model
model.compile(
loss = 'categorical_crossentropy',
optimizer = 'adam',
metrics = ['accuracy'])

In [ ]:
r = model.fit(train_generator,
                        validation_data = valid_generator,
                        epochs = 5,
                        steps_per_epoch = int(np.ceil(len(image_files) / batch_size)),
                       )

SyntaxError: ignored

In [ ]:
plt.plot(r.history['loss'], label = 'loss')
plt.plot(r.history['val_loss'], label = 'val loss')
plt.show()

In [ ]:
test_data_path = "/content/test"

In [ ]:
test_data = gen.flow_from_directory(
    test_data_path,
    target_size = img_size,
    batch_size = batch_size,
)

In [ ]:
predicted = model.predict(test_data)
predicted.shape